In [129]:
import pandas as pd

In [130]:
# preprocessing
dataset = pd.read_csv("insurance.csv")

In [131]:
dataset.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [132]:
# missing data
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [133]:
dataset['sex'].unique()

array(['female', 'male'], dtype=object)

In [134]:
# dataset['sex'] = dataset['sex'].map({'male': 0, 'female': 1}) # categorical variable 

In [135]:
dataset['sex'] = dataset['sex'].apply(lambda x:0 if x=='female' else 1 if x=='male' else -1) # categorical variable

In [136]:
dataset.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,yes,southwest,16884.92400
1,18,1,33.770,1,no,southeast,1725.55230
2,28,1,33.000,3,no,southeast,4449.46200
3,33,1,22.705,0,no,northwest,21984.47061
4,32,1,28.880,0,no,northwest,3866.85520


In [137]:
dataset['smoker'].unique()

array(['yes', 'no'], dtype=object)

In [138]:
dataset['smoker'] = dataset['smoker'].apply(lambda x: 0 if x=='no' else 1 if x=='yes' else -1) # categorical variable

In [139]:
dataset.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [140]:
dataset['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

In [141]:
# no relational order for region, so dummy variables are created
region_dummies = pd.get_dummies(dataset['region'], drop_first=True, dtype=int) # drop_first=True to avoid dummy variable trap
# dtype=int ensures the dummy variables are of integer type, 0 and 1, and not True/False

In [142]:
region_dummies.head()

,northwest,southeast,southwest
0,0,0,1
1,0,1,0
2,0,1,0
3,1,0,0
4,1,0,0


In [143]:
dataset = pd.concat([region_dummies, dataset], axis=1) # horizontal concatenation along y axis for columns

In [144]:
dataset.head()

,northwest,southeast,southwest,age,sex,bmi,children,smoker,region,charges
0,0,0,1,19,0,27.900,0,1,southwest,16884.92400
1,0,1,0,18,1,33.770,1,0,southeast,1725.55230
2,0,1,0,28,1,33.000,3,0,southeast,4449.46200
3,1,0,0,33,1,22.705,0,0,northwest,21984.47061
4,1,0,0,32,1,28.880,0,0,northwest,3866.85520


In [145]:
dataset.drop(['region'], axis=1, inplace=True) # drop the original region column

In [146]:
dataset.head()

,northwest,southeast,southwest,age,sex,bmi,children,smoker,charges
0,0,0,1,19,0,27.900,0,1,16884.92400
1,0,1,0,18,1,33.770,1,0,1725.55230
2,0,1,0,28,1,33.000,3,0,4449.46200
3,1,0,0,33,1,22.705,0,0,21984.47061
4,1,0,0,32,1,28.880,0,0,3866.85520


In [147]:
# X = dataset.drop(['charges'], axis=1) # features
X = dataset.iloc[:,:-1].values # all columns except the last one, .values() to make it an array
# y = dataset['charges'] # target variable
y = dataset.iloc[:,-1].values # last column, .values() to make it an array

In [148]:
X

array([[ 0.  ,  0.  ,  1.  , ..., 27.9 ,  0.  ,  1.  ],
       [ 0.  ,  1.  ,  0.  , ..., 33.77,  1.  ,  0.  ],
       [ 0.  ,  1.  ,  0.  , ..., 33.  ,  3.  ,  0.  ],
       ...,
       [ 0.  ,  1.  ,  0.  , ..., 36.85,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  1.  , ..., 25.8 ,  0.  ,  0.  ],
       [ 1.  ,  0.  ,  0.  , ..., 29.07,  0.  ,  1.  ]])

In [149]:
y

array([16884.924 ,  1725.5523,  4449.462 , ...,  1629.8335,  2007.945 ,
       29141.3603])

In [150]:
# Getting the training and test sets
from sklearn.model_selection import train_test_split   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # 80% training and 20% testing

In [151]:
# Gradient boosting algorithm don't need feature scaling

In [152]:
# building and training the model
import xgboost as xgb
model = xgb.XGBRegressor() # creating an instance of the XGBRegressor
model.fit(X_train, y_train) # fitting the model to the training data

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [153]:
# inference
y_pred = model.predict(X_test) # making predictions on the test set

In [154]:
y_pred[:5]

array([10083.838,  7742.762, 46423.664, 11340.579,  9828.638],
      dtype=float32)

In [155]:
y_test[:5]

array([ 9724.53   ,  8547.6913 , 45702.02235, 12950.0712 ,  9644.2525 ])

In [156]:
# Evaluating r-squared
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred) # R-squared score

In [157]:
r2

0.8670003485392761

In [158]:
# Evaluating adjusted r-squared
X_test.shape

(268, 8)

In [159]:
k = X_test.shape[1] # number of features
k

8

In [160]:
n = X_test.shape[0] # number of observations
n

268

In [161]:
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1) # adjusted r-squared formula
adjusted_r2

0.86289225119686

In [162]:
# k-fold cross validation
from sklearn.model_selection import cross_val_score
r2s = cross_val_score(estimator=model, X=X, y=y, scoring='r2', cv=10) # 10-fold cross validation
print(f"R-squared scores: {r2s}")

R-squared scores: [0.83939506 0.82024126 0.75780862 0.68220959 0.8485388  0.85285477
 0.86260279 0.78147487 0.82638456 0.84170074]


In [163]:
print(f"R-squared score: {[f'{score:.3f}' for score in r2s]}") # mean of the r-squared scores

R-squared score: ['0.839', '0.820', '0.758', '0.682', '0.849', '0.853', '0.863', '0.781', '0.826', '0.842']


In [164]:
print(f"Mean R-squared score: {r2s.mean():.3f}") # mean of the r-squared scores

Mean R-squared score: 0.811


In [165]:
print(f"Standard deviation of R-squared scores: {r2s.std():.3f}") # standard deviation of the r-squared scores

Standard deviation of R-squared scores: 0.053


In [166]:
# Improved model
model_improved = xgb.XGBRegressor(n_estimators=1000, # Increased number of estimators, many trees but with small learning rate
                                  learning_rate=0.05, # shrinkage to prevent overfitting
                                  max_depth=4, # shallow trees to prevent overfitting
                                  min_child_weight=5, # regularization: minimum sum of instance weight (hessian) in a child
                                  subsample=0.8, # (row sampling) regularization: subsample ratio of the training instances
                                  colsample_bytree=0.8, # (column sampling) regularization: subsample ratio of features when constructing each tree
                                  gamma=0.1, # regularization: minimum loss reduction required to make a further partition on a leaf node (split)
                                  reg_alpha=0.1, # L1 regularization term on weights
                                  reg_lambda=1.0, # L2 regularization term on weights
                                  random_state=42,
                                  early_stopping_rounds=50, # rounds without improvement after which training will be stopped
                                  )
model_improved.fit(X_train, 
                   y_train,
                   eval_set=[(X_test, y_test)], # evaluation set for early stopping
                   verbose=10 # output evaluation metric every 10 rounds
                   )


[0]	validation_0-rmse:12063.88412
[10]	validation_0-rmse:8759.06325
[20]	validation_0-rmse:7014.57232
[30]	validation_0-rmse:5702.24856
[40]	validation_0-rmse:4875.83940
[50]	validation_0-rmse:4526.27114
[60]	validation_0-rmse:4255.45115
[70]	validation_0-rmse:4155.11927
[80]	validation_0-rmse:4080.21397
[90]	validation_0-rmse:4043.42275
[100]	validation_0-rmse:4025.58783
[110]	validation_0-rmse:4025.38410
[120]	validation_0-rmse:4034.68605
[130]	validation_0-rmse:4057.81674
[140]	validation_0-rmse:4075.71000
[150]	validation_0-rmse:4086.56990
[160]	validation_0-rmse:4093.96923


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,50
,enable_categorical,False
,eval_metric,None


In [167]:
# inference
y_pred_improved = model_improved.predict(X_test) # making predictions on the test set

In [168]:
# Evaluating r-squared
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred_improved) # R-squared score
r2

0.8983649439678434

In [169]:
# Hyperparameter Search
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [500, 1000, 2000],
    'subsample': [0.7, 0.8, 1],
    'colsample_bytree': [0.7, 0.8, 1],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [1, 1.5, 2]
}

search = RandomizedSearchCV(
    estimator=xgb.XGBRegressor(random_state=42), # specify the model
    param_distributions=param_grid, # specify the parameter grid
    n_iter=100, # number of iterations
    scoring='r2', # scoring metric
    cv=3, # cross-validation splitting strategy
    verbose=3, # verbosity level
    random_state=42, # random state for reproducibility
    n_jobs=-1 # use all available cores
)

search.fit(X_train, y_train) # fit the search to the training data
print(f"Best parameters: {search.best_params_}") # print the best parameters found

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters: {'subsample': 0.8, 'reg_lambda': 2, 'reg_alpha': 0.1, 'n_estimators': 500, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 1}


In [170]:
print(search.best_estimator_) # print the best score found

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=0.2, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, ...)


In [171]:
y_pred_hyper = search.predict(X_test) # make predictions on the test set

In [172]:
# Evaluating r-squared
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred_hyper) # R-squared score
r2

0.9042474840822391

In [173]:
# Manually initializing the model with the best parameters
best_model = xgb.XGBRegressor(**search.best_params_, random_state=42)
best_model.fit(X_train, y_train)


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,1
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [174]:
# predicting on the test set
y_pred_best = best_model.predict(X_test)


In [175]:
# Evaluating r-squared
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred_best) # R-squared score
r2

0.9042474840822391

In [176]:
# k-fold cross validation
from sklearn.model_selection import cross_val_score
r2s_new = cross_val_score(estimator=best_model, X=X, y=y, scoring='r2', cv=10) # 10-fold cross validation
print(f"R-squared scores: {r2s_new}")

R-squared scores: [0.89600889 0.87073283 0.84846391 0.76866608 0.86854793 0.93049972
 0.88151199 0.82257525 0.87912844 0.86531355]


In [177]:
print(f"Mean R-squared score: {r2s_new.mean():.3f}") # mean of the r-squared scores

Mean R-squared score: 0.863


In [178]:
r2s.std(), r2s_new.std()

(0.05308646345724576, 0.04133846222342994)

In [179]:
r2s.mean(), r2s_new.mean()

(0.8113211059386878, 0.863144858086019)